In [1]:
!pip install pdfplumber
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.9 MB/s eta 0:00:00


In [24]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving PM Peak External.pdf to PM Peak External.pdf


# Run

In [25]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re
import fitz
import shutil

In [26]:
output_folder = "output_pages"
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

count = 0
os.makedirs(output_folder, exist_ok=True)
for filename in uploaded.keys():
    doc = fitz.open(filename)
    for i, page in enumerate(doc):
        text = page.get_text("text")
        lines = text.splitlines()
        if lines and "HCM Signalized Intersection Capacity Analysis" in lines[0]:
            new_doc = fitz.open()
            new_doc.insert_pdf(doc, from_page=i, to_page=i)
            output_path = os.path.join(output_folder, f"HCM_Signalized_{count+1}.pdf")
            new_doc.save(output_path)
            new_doc.close()
            count += 1

In [27]:
def get_nth_numeric_value(data, n):
    numeric_values = []
    for item in data:
        try:
            num = float(item)
            numeric_values.append(num)
        except (ValueError, TypeError):
            continue
    return numeric_values[n - 1] if len(numeric_values) >= n else None

def extract_lines_below_pattern(lines, pattern, offset=2, range_length=1):
    for i, line in enumerate(lines):
        if pattern.strip().lower() in line.strip().lower():
            start_index = i + offset
            end_index = start_index + range_length
            return [l.strip() for l in lines[start_index:end_index] if start_index < len(lines)]
    return []

In [28]:
Output = pd.DataFrame(columns= ['Scenario','Node','V/C', 'Delay', 'LOS', 'W_Movement','W_Delay','W_V/C','W_LOS'] )
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    all_text = []
    with pdfplumber.open(file_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            text = page.extract_text( layout=True)
            if text:
                all_text.append(text)
    full_text = '\n'.join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)

    node= data[4][9:30]
    Scenario= data[4][32:]
    stripped_lines = extract_lines_below_pattern(data, "Intersection Summary", offset=1, range_length=2)
    abc = stripped_lines[0].split(" ")
    control_delay = get_nth_numeric_value(abc, 2)
    level_of_service = stripped_lines[0].split(" ")[-1]
    v_c_ratio = stripped_lines[1].split(" ")[-1]
    line_1 = extract_lines_below_pattern(data, "Movement", offset=0, range_length=1)
    line_2 = extract_lines_below_pattern(data, "Intersection Summary", offset=-8, range_length=6)
    lines = line_1 + line_2
    ref = [i for i in range(len(lines[0][17:])) if (i == 0 or lines[0][17:][i-1] == ' ') and lines[0][17:][i] != ' ']
    df_row = []
    for k in [0,1,-2,-1]:
        starts = [i for i in range(len(lines[k][17:])) if (i == 0 or lines[k][17:][i-1] == ' ') and lines[k][17:][i] != ' ']
        text = lines[k][17:]
        values = []
        for i, start in enumerate(starts):
            end = starts[i + 1] if i + 1 < len(starts) else len(text)
            val = text[start:end].strip()
            values.append((start, val))
        row = ['' for _ in range(len(ref))]
        for start_idx, val in values:
            for i, r in enumerate(ref):
                if abs(start_idx - r) <= 2:
                    row[i] = val
                    break
        df_row.append(row)
    df = pd.DataFrame(df_row, columns=df_row[0])
    try:
        delay_row = pd.to_numeric(df.iloc[2], errors='coerce')
        max_col = delay_row.idxmax()
        vc = df.loc[1, max_col] if 1 in df.index and max_col in df.columns else 'NA'
        delay = df.loc[2, max_col] if 2 in df.index and max_col in df.columns else 'NA'
        los = df.loc[3, max_col] if 3 in df.index and max_col in df.columns else 'NA'

    except (IndexError, KeyError, ValueError):
        vc = delay = los = 'NA'
    new_row = {
        'Scenario': Scenario,
        'Node': node,
        'V/C':v_c_ratio,
        'Delay': control_delay,
        'LOS': level_of_service,
        'W_Movement': max_col,
        'W_Delay': delay,
        'W_V/C': vc,
        'W_LOS': los
    }

    Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)

/tmp/ipython-input-28-1162201972.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Output = pd.concat([Output, pd.DataFrame([new_row])], ignore_index=True)


# change Output file Name as per your choice "Synchro_Report.xlsx"

In [29]:
Output

,Scenario,Node,V/C,Delay,LOS,W_Movement,W_Delay,W_V/C,W_LOS
0,...,5001:,0.59,10.7,B,NWT,19.4,0.12,B
1,...,4000:,1.44,163.1,F,NBL,569.6,2.17,F
2,...,2000:,0.13,16.5,B,WBT,23.6,0.12,C
3,...,1000:,0.45,22.3,C,WBL,30.8,0.68,C


In [30]:
Output.to_excel('Synchro_Report.xlsx', index=False)